In [1]:
import os 
import math
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from numpy import count_nonzero

In [2]:
dir_ = './20'
g_type = 'h'
p = pd.read_pickle(os.path.join(dir_, g_type + '_p.pkl'))
r = pd.read_pickle(os.path.join(dir_, g_type + '_r.pkl'))
n = pd.read_pickle(os.path.join(dir_, g_type + '_n.pkl'))

In [3]:
# Choose lambda
opt_lambdas = []
for round_ in range(5):
    g_lambdas = []
    for g_size in range(2,9):
        train = p[p['group_size']==g_size]
        num_group = len(train)/11
        train = train[(train['gid']<int(num_group/5*round_)) | ((train['gid']>=int(num_group/5*(round_+1))))]
        
        p_lambdas = []
        for lambda_ in [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
            p_lambda = train[train['lambda']==lambda_]
            p_lambdas.append(sum(p_lambda['values'].values)/len(p_lambda['values'].values))
        g_lambdas.append(p_lambdas.index(max(p_lambdas))/10)
    opt_lambdas.append(g_lambdas)

In [4]:
opt_lambdas

[[0.6, 0.5, 0.4, 0.4, 0.4, 0.4, 0.4],
 [0.6, 0.5, 0.5, 0.4, 0.4, 0.4, 0.4],
 [0.6, 0.5, 0.5, 0.5, 0.5, 0.4, 0.4],
 [0.6, 0.5, 0.5, 0.4, 0.4, 0.4, 0.4],
 [0.6, 0.5, 0.5, 0.4, 0.4, 0.5, 0.4]]

In [5]:
# opt_lambdas = []
# for i in range(5):
#     opt_lambdas.append(list(np.ones(7)))
# opt_lambdas

In [6]:
# Evaluation
p_list = []
r_list = []
n_list = []
for round_ in range(5):
    g_lambdas = opt_lambdas[round_]
    ps = []
    rs = []
    ns = []
    for g_size in range(2,9):
        lambda_ = g_lambdas[g_size-2]
        test_p = p[p['group_size']==g_size]
        test_r = r[r['group_size']==g_size]
        test_n = n[n['group_size']==g_size]
        num_group = len(test_p)/11
        test_p = test_p[(test_p['gid']>=int(num_group/5*round_)) & ((test_p['gid']<int(num_group/5*(round_+1))))]
        test_r = test_r[(test_r['gid']>=int(num_group/5*round_)) & ((test_r['gid']<int(num_group/5*(round_+1))))]
        test_n = test_n[(test_n['gid']>=int(num_group/5*round_)) & ((test_n['gid']<int(num_group/5*(round_+1))))]
        
        test_p = test_p[test_p['lambda']==lambda_]['values'].values
        test_r = test_r[test_r['lambda']==lambda_]['values'].values
        test_n = test_n[test_n['lambda']==lambda_]['values'].values
        ps.append(sum(test_p)/len(test_p))
        rs.append(sum(test_r)/len(test_r))
        ns.append(sum(test_n)/len(test_n))
    p_list.append(ps)
    r_list.append(rs)
    n_list.append(ns)

In [7]:
p_avg = []
r_avg = []
n_avg = []
for g_size in range(7):
    precision = 0
    recall = 0
    nDCG = 0
    for round_ in range(5):
        precision += p_list[round_][g_size]
        recall += r_list[round_][g_size]
        nDCG += n_list[round_][g_size]
    p_avg.append(precision/5)
    r_avg.append(recall/5)
    n_avg.append(nDCG/5)

In [8]:
p_avg

[0.20444896115627825,
 0.185326536643026,
 0.16949999999999998,
 0.17375757575757575,
 0.16607230392156863,
 0.16872841444270015,
 0.17]

In [9]:
print(sum(p_avg)/len(p_avg))
print(sum(r_avg)/len(r_avg))
print(sum(n_avg)/len(n_avg))

0.17683339884587837
0.00446654659370285
0.09352047142247842
